In [1]:
%pip install --quiet --upgrade langchain langchain_community langchain-chroma pypdf langchain_text_splitters langchain_cohere tenacity python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

In [2]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("kanoni.pdf")
pages = loader.load()
text = ""
for page in pages:
    text += page.page_content

# print first 1000 characters



incorrect startxref pointer(1)
parsing for Object Streams


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    separators=["თავი [IVXLCDM]+", "მუხლი \d+"],
    is_separator_regex=True,
)
chunks = text_splitter.split_text(text)



In [4]:
# embed chunks
from langchain_cohere import CohereEmbeddings
import os
# store chunks in chroma
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="kanoni",
    embedding_function=CohereEmbeddings(cohere_api_key=os.environ["COHERE_API_KEY"], model="embed-multilingual-light-v3.0"),
    persist_directory="./chroma_langchain_db",
    create_collection_if_not_exists=True
)



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Halvadar\AppData\Local\sagemaker\sagemaker\config.yaml


In [5]:
import os
import time
from tenacity import retry, wait_exponential, stop_after_attempt
@retry(wait=wait_exponential(multiplier=1, min=4, max=10), stop=stop_after_attempt(5))
def get_embeddings(texts):
    embeddings = CohereEmbeddings(cohere_api_key=os.environ["COHERE_API_KEY"], model="embed-multilingual-light-v3.0", )
    return embeddings.embed(texts=texts,input_type="search_document")



# Add texts in smaller batches with delay
batch_size = 5  # Reduced batch size
total_batches = len(chunks) // batch_size + (1 if len(chunks) % batch_size > 0 else 0)

for i in range(0, len(chunks), batch_size):
    try:
        print(f"Processing batch {i//batch_size + 1} of {total_batches}")
        batch = chunks[i:i+batch_size]
        print("embedding batch")
        # Get embeddings for the batch
        embeddings = get_embeddings(batch)
        print("adding to vector store")
        # Add texts and embeddings to the vector store
        vector_store.add_texts(texts=batch, embeddings=embeddings)
        
        print(f"Successfully added batch {i//batch_size + 1}")
        time.sleep(1)  # Increased delay between batches
    except Exception as e:
        print(f"Error processing batch {i//batch_size + 1}: {str(e)}")
        time.sleep(2)  # Longer delay if an error occurs

print("Finished processing all batches")


Processing batch 1 of 103
embedding batch
adding to vector store
Successfully added batch 1
Processing batch 2 of 103
embedding batch
adding to vector store
Successfully added batch 2
Processing batch 3 of 103
embedding batch
adding to vector store
Successfully added batch 3
Processing batch 4 of 103
embedding batch
adding to vector store
Successfully added batch 4
Processing batch 5 of 103
embedding batch
adding to vector store
Successfully added batch 5
Processing batch 6 of 103
embedding batch
adding to vector store
Successfully added batch 6
Processing batch 7 of 103
embedding batch
adding to vector store
Successfully added batch 7
Processing batch 8 of 103
embedding batch
adding to vector store
Successfully added batch 8
Processing batch 9 of 103
embedding batch
adding to vector store
Successfully added batch 9
Processing batch 10 of 103
embedding batch
adding to vector store
Successfully added batch 10
Processing batch 11 of 103
embedding batch
adding to vector store
Successfully

: 

In [46]:
# Add the following code to retrieve the first embedding
results = vector_store.similarity_search(query="ამოღებულია")

# Print the first embedding
print(results[0].page_content)

# ... existing code ...

generates a list of rectangular boxes for the target content regions. Diﬀerent
from traditional methods, it relies on deep convolutional neural networks rather
than manually curated rules to identify content regions. It is formulated as an
object detection problem and state-of-the-art models like Faster R-CNN [ 28] and
Mask R-CNN [ 12] are used. This yields prediction results of high accuracy and
makes it possible to build a concise, generalized interface for layout detection.
LayoutParser , built upon Detectron2 [ 35], provides a minimal API that can
perform layout detection with only four lines of code in Python:
1import layoutparser as lp
2image = cv2 . imread (" image_file ") # load images
3model = lp. Detectron2LayoutModel (
4 "lp :// PubLayNet / faster_rcnn_R_50_FPN_3x / config ")
5layout = model . detect ( image )
LayoutParser provides a wealth of pre-trained model weights using various
datasets covering diﬀerent languages, time periods, and document types. Due to
